In [70]:
from pymongo import MongoClient
from bson.json_util import dumps
import pymongo
import pandas as pd
import requests

mongodbURL = "mongodb://localhost/Chat_Sentiment_Analysis_Service"
client = MongoClient(mongodbURL, connectTimeoutMS=2000,serverSelectionTimeoutMS=2000)
db = client.get_database()

We are going to use 

In [74]:
shakespeare=pd.read_csv("input/Shakespeare_data.csv").drop(columns=["PlayerLinenumber","ActSceneLine","Dataline"],axis=1).dropna()

shakespeare["Play"]=shakespeare["Play"].apply(lambda x: x.replace(" ",""))
shakespeare["Player"]=shakespeare["Player"].apply(lambda x: x.replace(" ",""))
shakespeare["PlayerLine"]=shakespeare["PlayerLine"].apply(lambda x: x.replace(" ","%20"))



shakespeare_plays=shakespeare["Play"].unique()

shakespeare_players=shakespeare["Player"].unique()

players_in_play=[{play:shakespeare[shakespeare["Play"]==play]["Player"].unique()} for play in shakespeare_plays]

Feeding users database

In [75]:
for name in shakespeare_players:
    a=requests.get(f"http://127.0.0.1:4000/user/create?username={name}").text
    print(a)

{
  "kinghenryiv": "added to users list!"
}

{
  "westmoreland": "added to users list!"
}

{
  "falstaff": "added to users list!"
}

{
  "princehenry": "added to users list!"
}

{
  "poins": "added to users list!"
}

{
  "earlofworcester": "added to users list!"
}

{
  "northumberland": "added to users list!"
}

{
  "hotspur": "added to users list!"
}

{
  "sirwalterblunt": "added to users list!"
}

{
  "firstcarrier": "added to users list!"
}

{
  "ostler": "added to users list!"
}

{
  "secondcarrier": "added to users list!"
}

{
  "gadshill": "added to users list!"
}

{
  "chamberlain": "added to users list!"
}

{
  "bardolph": "added to users list!"
}

{
  "peto": "added to users list!"
}

{
  "firsttraveller": "added to users list!"
}

{
  "thieves": "added to users list!"
}

{
  "travellers": "added to users list!"
}

{
  "ladypercy": "added to users list!"
}

{
  "servant": "added to users list!"
}

{
  "francis": "added to users list!"
}

{
  "vintner": "added to users list!"
}

{
  "secondattendant": "added to users list!"
}

{
  "octaviuscaesar": "added to users list!"
}

{
  "lepidus": "added to users list!"
}

{
  "mardian": "added to users list!"
}

{
  "pompey": "added to users list!"
}

{
  "menecrates": "added to users list!"
}

{
  "menas": "added to users list!"
}

{
  "varrius": "added to users list!"
}

{
  "mecaenas": "added to users list!"
}

{
  "agrippa": "added to users list!"
}

{
  "octavia": "added to users list!"
}

{
  "attendants": "added to users list!"
}

{
  "firstservant": "added to users list!"
}

{
  "secondservant": "added to users list!"
}

{
  "ventidius": "added to users list!"
}

{
  "silius": "added to users list!"
}

{
  "eros": "added to users list!"
}

{
  "canidius": "added to users list!"
}

{
  "taurus": "added to users list!"
}

{
  "scarus": "added to users list!"
}

{
  "dolabella": "added to users list!"
}

{
  "euphronius": "added to users list!"
}

{
  "thyreus": "added to users list!"
}

{
  "thirdsoldier": "adde

{
  "ladyfaulconbridge": "added to users list!"
}

{
  "gurney": "added to users list!"
}

{
  "lewis": "added to users list!"
}

{
  "arthur": "added to users list!"
}

{
  "austria": "added to users list!"
}

{
  "constance": "added to users list!"
}

{
  "kingphilip": "added to users list!"
}

{
  "blanch": "added to users list!"
}

{
  "frenchherald": "added to users list!"
}

{
  "englishherald": "added to users list!"
}

{
  "cardinalpandulph": "added to users list!"
}

{
  "elinor": "added to users list!"
}

{
  "hubert": "added to users list!"
}

{
  "firstexecutioner": "added to users list!"
}

{
  "pembroke": "added to users list!"
}

{
  "bigot": "added to users list!"
}

{
  "melun": "added to users list!"
}

{
  "flavius": "added to users list!"
}

{
  "firstcommoner": "added to users list!"
}

{
  "marullus": "added to users list!"
}

{
  "secondcommoner": "added to users list!"
}

{
  "caesar": "added to users list!"
}

{
  "casca": "added to users list!"
}

{
  "calpurn

{
  "thaliard": "added to users list!"
}

{
  "helicanus": "added to users list!"
}

{
  "cleon": "added to users list!"
}

{
  "dionyza": "added to users list!"
}

{
  "firstfisherman": "added to users list!"
}

{
  "secondfisherman": "added to users list!"
}

{
  "thirdfisherman": "added to users list!"
}

{
  "simonides": "added to users list!"
}

{
  "thaisa": "added to users list!"
}

{
  "knights": "added to users list!"
}

{
  "marshal": "added to users list!"
}

{
  "firstknight": "added to users list!"
}

{
  "message": "This user already exists in database", 
  "status": "error"
}

{
  "escanes": "added to users list!"
}

{
  "secondknight": "added to users list!"
}

{
  "thirdknight": "added to users list!"
}

{
  "lychorida": "added to users list!"
}

{
  "secondsailor": "added to users list!"
}

{
  "cerimon": "added to users list!"
}

{
  "philemon": "added to users list!"
}

{
  "leonine": "added to users list!"
}

{
  "marina": "added to users list!"
}

{
  "firstpirate

{
  "margarelon": "added to users list!"
}

{
  "myrmidons": "added to users list!"
}

{
  "dukeorsino": "added to users list!"
}

{
  "curio": "added to users list!"
}

{
  "valentine": "added to users list!"
}

{
  "viola": "added to users list!"
}

{
  "sirtobybelch": "added to users list!"
}

{
  "sirandrew": "added to users list!"
}

{
  "olivia": "added to users list!"
}

{
  "malvolio": "added to users list!"
}

{
  "fabian": "added to users list!"
}

{
  "proteus": "added to users list!"
}

{
  "speed": "added to users list!"
}

{
  "julia": "added to users list!"
}

{
  "lucetta": "added to users list!"
}

{
  "panthino": "added to users list!"
}

{
  "silvia": "added to users list!"
}

{
  "launce": "added to users list!"
}

{
  "thurio": "added to users list!"
}

{
  "firstoutlaw": "added to users list!"
}

{
  "secondoutlaw": "added to users list!"
}

{
  "thirdoutlaw": "added to users list!"
}

{
  "eglamour": "added to users list!"
}

{
  "outlaws": "added to users list!"

Creating conversations (plays)

In [76]:
for play in shakespeare_plays:
    RES=requests.get(f"http://127.0.0.1:4000/chat/create?conversation={play}").text
    print(RES)

{
  "HenryIV": "created!"
}

{
  "HenryVIPart1": "created!"
}

{
  "HenryVIPart2": "created!"
}

{
  "HenryVIPart3": "created!"
}

{
  "Allswellthatendswell": "created!"
}

{
  "Asyoulikeit": "created!"
}

{
  "AntonyandCleopatra": "created!"
}

{
  "AComedyofErrors": "created!"
}

{
  "Coriolanus": "created!"
}

{
  "Cymbeline": "created!"
}

{
  "Hamlet": "created!"
}

{
  "HenryV": "created!"
}

{
  "HenryVIII": "created!"
}

{
  "KingJohn": "created!"
}

{
  "JuliusCaesar": "created!"
}

{
  "KingLear": "created!"
}

{
  "LovesLaboursLost": "created!"
}

{
  "macbeth": "created!"
}

{
  "Measureformeasure": "created!"
}

{
  "MerchantofVenice": "created!"
}

{
  "MerryWivesofWindsor": "created!"
}

{
  "AMidsummernightsdream": "created!"
}

{
  "MuchAdoaboutnothing": "created!"
}

{
  "Othello": "created!"
}

{
  "Pericles": "created!"
}

{
  "RichardII": "created!"
}

{
  "RichardIII": "created!"
}

{
  "RomeoandJuliet": "created!"
}

{
  "TamingoftheShrew": "created!"
}

{
  "The

Adding each character to its play

In [77]:
for play in players_in_play:
        for k,v in play.items():
            for player in v:
                requests.get(f"http://127.0.0.1:4000/chat/{k}/adduser?username={player}")

Adding sentence of each character to its play

In [78]:
for index in shakespeare.index:
    requests.get(f"http://127.0.0.1:4000/chat/{shakespeare.loc[index]['Play']}/addmessage?username={shakespeare.loc[index]['Player']}&message={shakespeare.loc[index]['PlayerLine']}")

In [69]:
db.messages.remove({})
db.users.remove({})
db.conversations.remove({})

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  """Entry point for launching an IPython kernel.
/usr/lib/python3/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  
/usr/lib/python3/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  This is separate from the ipykernel package so we can avoid doing imports until


{'n': 36, 'ok': 1.0}